In [8]:
''' Team Name- CODECRAFTERS
    Team Members- Mayank Srivastava(Leader)
                  Suhani Gupta
                  Ishan Sharma
                  Rishabh
    Approach:
    We used ridge regression and K fold setup for calculating the SMAPE'''

' Team Name- CODECRAFTERS\n    Team Members- Mayank Srivastava(Leader)\n                  Suhani Gupta\n                  Ishan Sharma\n                  Rishabh\n    Approach:\n    We used ridge regression and K fold setup for calculating the SMAPE'

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error

In [2]:
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return 100 * np.mean(diff)
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
print("Train: ",train.head())
print("Test: ",test.head())

Train:     sample_id                                    catalog_content  \
0      33127  Item Name: La Victoria Green Taco Sauce Mild, ...   
1     198967  Item Name: Salerno Cookies, The Original Butte...   
2     261251  Item Name: Bear Creek Hearty Soup Bowl, Creamy...   
3      55858  Item Name: Judee’s Blue Cheese Powder 11.25 oz...   
4     292686  Item Name: kedem Sherry Cooking Wine, 12.7 Oun...   

                                          image_link  price  
0  https://m.media-amazon.com/images/I/51mo8htwTH...   4.89  
1  https://m.media-amazon.com/images/I/71YtriIHAA...  13.12  
2  https://m.media-amazon.com/images/I/51+PFEe-w-...   1.97  
3  https://m.media-amazon.com/images/I/41mu0HAToD...  30.34  
4  https://m.media-amazon.com/images/I/41sA037+Qv...  66.49  
Test:     sample_id                                    catalog_content  \
0     100179  Item Name: Rani 14-Spice Eshamaya's Mango Chut...   
1     245611  Item Name: Natural MILK TEA Flavoring extract ...   
2     146

In [3]:
train = train.dropna(subset=["catalog_content", "price"])
test["catalog_content"] = test["catalog_content"].fillna("")
print("Train cleaned: ",train.head())
print("Test cleaned: ",test.head())

Train cleaned:     sample_id                                    catalog_content  \
0      33127  Item Name: La Victoria Green Taco Sauce Mild, ...   
1     198967  Item Name: Salerno Cookies, The Original Butte...   
2     261251  Item Name: Bear Creek Hearty Soup Bowl, Creamy...   
3      55858  Item Name: Judee’s Blue Cheese Powder 11.25 oz...   
4     292686  Item Name: kedem Sherry Cooking Wine, 12.7 Oun...   

                                          image_link  price  
0  https://m.media-amazon.com/images/I/51mo8htwTH...   4.89  
1  https://m.media-amazon.com/images/I/71YtriIHAA...  13.12  
2  https://m.media-amazon.com/images/I/51+PFEe-w-...   1.97  
3  https://m.media-amazon.com/images/I/41mu0HAToD...  30.34  
4  https://m.media-amazon.com/images/I/41sA037+Qv...  66.49  
Test cleaned:     sample_id                                    catalog_content  \
0     100179  Item Name: Rani 14-Spice Eshamaya's Mango Chut...   
1     245611  Item Name: Natural MILK TEA Flavoring extract 

In [4]:
vectorizer = TfidfVectorizer(
    max_features=50000, 
    ngram_range=(1, 2),
    stop_words='english'
)
model = Ridge(alpha=1.0, random_state=42)

In [5]:
# K-Fold setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros(len(train))
test_preds = np.zeros(len(test))

X = train["catalog_content"].astype(str)
y = train["price"].astype(float)
X_test = test["catalog_content"].astype(str)

for fold, (trn_idx, val_idx) in enumerate(kf.split(X, y)):
    print(f"FOLD {fold+1}/5")
    
    X_train, X_val = X.iloc[trn_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[trn_idx], y.iloc[val_idx]
    
    pipe = make_pipeline(vectorizer, Ridge(alpha=1.0, random_state=42))
    pipe.fit(X_train, y_train)
    
    val_pred = pipe.predict(X_val)
    oof_preds[val_idx] = val_pred
    test_preds += pipe.predict(X_test) / kf.n_splits

score = smape(y, oof_preds)
print(f"\nOOF SMAPE: {score:.2f}%")

FOLD 1/5
FOLD 2/5
FOLD 3/5
FOLD 4/5
FOLD 5/5

OOF SMAPE: 69.12%


In [6]:
sub = pd.DataFrame({
    "sample_id": test["sample_id"],
    "price": np.maximum(0, test_preds)  # Ensure positive prices
})
sub.to_csv("test_out.csv", index=False)

print("\n✅ test_out.csv generated successfully!")
print(sub.head())


✅ test_out.csv generated successfully!
   sample_id      price
0     100179  26.607213
1     245611  26.111775
2     146263  23.361750
3      95658   0.627698
4      36806  65.854633
